<a href="https://colab.research.google.com/github/brandontan99/Self_Love_App/blob/master/OneVsRest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/brandontan99/Self_Love_App.git
%cd Self_Love_App

Cloning into 'Self_Love_App'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 166 (delta 91), reused 51 (delta 22), pack-reused 0
Receiving objects: 100% (166/166), 1.81 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (91/91), done.
/content/Self_Love_App


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from data_cleaning import *
from Data_Normalization import *
import time
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
import sklearn
import statistics
import joblib
from copy import deepcopy
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split
from feature_selection import *

In [10]:
    # to do the chi2 analysis
    df = pd.read_csv("WID3006 ML Questionnaire.csv")
    df = data_cleaning(df)
    # label encoding is used instead of one-hot encoding because one-hot encoding will create many features for one question
    # but label encoding is able to perform chi-square test between a single question as a whole and the hobby  
    x_df, y_df = label_encoding(df)
    # only evaluate on train data to prevent data leakage
    x_train_label_encoded, _, y_train, _ = train_test_split(x_df.to_numpy(), y_df.to_numpy(), test_size=0.2, random_state=1)
    chi2_result = chi2_analysis(x_train_label_encoded, x_df, y_train)

    # to do Recursive Feature Elimination(rfe) feature selection
    df = pd.read_csv("WID3006 ML Questionnaire.csv")
    df = data_cleaning(df)
    df = data_encoding(df)
    df_norm = data_normalization(df)
    x = df_norm.iloc[:, :64]
    y = df_norm.iloc[:, 64:]
    x_numpy, y_numpy = x.to_numpy(), y.to_numpy()
    x_train, x_test, y_train, y_test = train_test_split(x_numpy, y_numpy, test_size=0.2, random_state=1)
    rfe_result = rfe_cv(x_train, y_train, x.columns, y.columns, LogisticRegression())

    # to pick the selected best questions as features
    # Eg: choose the best 13 features as below
    df = pd.read_csv("WID3006 ML Questionnaire.csv")
    df = data_cleaning(df)
    df = data_encoding(df)
    df_norm = data_normalization(df)
    # choose either chi2 or rfe 
    best_k_features = select_best_k_features(chi2_result, k=13) # choose chi2_result 
    best_k_features = select_best_k_features(rfe_result, k=13) # choose rfe_analysis
    x = filter_features(best_k_features, df_norm)


Hobby: Sports and Outdoors
Best number of features: 11
Score: 0.6797849462365592
1 Gender: _Male
21 Choose a pet which you prefer to keep._Horse
25 Choose a pet which you prefer to keep._Tortoise
40 What is your favorite color?_Blue
50 What is your learning style? (Pick one that benefit you the most)_By reading an e-Book
52 Do you enjoy challenges?
55 Would you prefer to engage your brain more than your body?
57 Are you a perfectionist?
58 Are you a trusting person?
59 Do you have lot of patience?
64 How confident are you in your own abilities?

Hobby: Games
Best number of features: 10
Score: 0.5944086021505377
1 Gender: _Male
4 What is your current occupation?_University student
11 I prefer to spend my money on...._The latest fashion
22 Choose a pet which you prefer to keep._I'm not a pet person
31 What do you worry more about the most?_Money
33 What do you worry more about the most?_Your family and friends
35 When you retire, you'd like to live..._Exactly where I live now
42 What is

In [4]:
def trainModel(pipeline,x,y):
  f1_list=[]
  pipelineList=[]
  x_train, x_test,y_train,y_test = train_test_split(x,y, random_state=1, test_size=0.25, shuffle=True)
  x_test=x_test.iloc[20:,:]
  y_test=y_test.iloc[20:,:]
  for i,hobby in enumerate(y.columns):
      # Training logistic regression model on train data
      pipelineList.append(deepcopy(pipeline).fit(x_train, y_train[hobby]))
    
      # calculating test accuracy
      prediction = pipelineList[i].predict(x_test)
      f1=sklearn.metrics.f1_score(y_test[hobby], prediction,average="micro")
      f1_list.append(f1)
      print("F1 score for",hobby,": ",f1)
  print('Average f1-score: {}% ({})'.format(statistics.mean(f1_list)*100,statistics.stdev(f1_list)*100))
  return pipelineList

def predict(pipelineList,x_pred):
  x_pred=np.array(x_pred).reshape(-1,x.shape[1])
  pred=[]
  for i,hobby in enumerate(y.columns):
    prediction = pipelineList[i].predict(x_pred)
    if prediction[0]==1:
      pred.append(hobby)
  print(pred)

In [5]:
# Using pipeline for applying logistic regression and one vs rest classifier
OneVsRestClassifier.multilabel_=True
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
NB_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
SVC_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

In [11]:
Logreg_pipelineList=trainModel(LogReg_pipeline,x,y)
for i in range(5): 
  predict(Logreg_pipelineList,x.iloc[i,:])

F1 score for Sports and Outdoors :  0.6428571428571429
F1 score for Games :  0.5
F1 score for Spiritual and Mental :  0.6428571428571429
F1 score for Performing Arts :  0.6785714285714286
F1 score for Arts and Craft :  0.6785714285714286
F1 score for Food and Drinks :  0.7857142857142857
F1 score for Collecting :  0.9642857142857143
F1 score for Rejuvenation :  0.8214285714285714
Average f1-score: 71.42857142857143% (14.028292943742365)
['Sports and Outdoors', 'Games', 'Spiritual and Mental']
['Sports and Outdoors', 'Games']
['Sports and Outdoors', 'Games', 'Spiritual and Mental', 'Performing Arts']
['Sports and Outdoors', 'Games', 'Spiritual and Mental']
['Sports and Outdoors', 'Games', 'Spiritual and Mental']


In [12]:
NB_pipelineList=trainModel(NB_pipeline,x,y)
for i in range(5): 
  predict(NB_pipelineList,x.iloc[i,:])

F1 score for Sports and Outdoors :  0.5714285714285714
F1 score for Games :  0.5714285714285714
F1 score for Spiritual and Mental :  0.6428571428571429
F1 score for Performing Arts :  0.6785714285714286
F1 score for Arts and Craft :  0.75
F1 score for Food and Drinks :  0.75
F1 score for Collecting :  0.9285714285714286
F1 score for Rejuvenation :  0.8214285714285714
Average f1-score: 71.42857142857143% (12.371791482634837)
['Sports and Outdoors', 'Games', 'Spiritual and Mental']
['Sports and Outdoors']
['Sports and Outdoors', 'Games', 'Spiritual and Mental', 'Performing Arts']
['Sports and Outdoors', 'Games', 'Spiritual and Mental']
['Sports and Outdoors', 'Games', 'Spiritual and Mental', 'Food and Drinks']


In [13]:
SVC_pipelineList=trainModel(SVC_pipeline,x,y)
for i in range(5): 
  predict(SVC_pipelineList,x.iloc[i,:])

F1 score for Sports and Outdoors :  0.6428571428571429
F1 score for Games :  0.4642857142857143
F1 score for Spiritual and Mental :  0.6428571428571429
F1 score for Performing Arts :  0.6785714285714286
F1 score for Arts and Craft :  0.6428571428571429
F1 score for Food and Drinks :  0.8214285714285714
F1 score for Collecting :  0.8928571428571429
F1 score for Rejuvenation :  0.7142857142857143
Average f1-score: 68.75% (12.912296668207922)
['Sports and Outdoors', 'Games', 'Spiritual and Mental']
['Sports and Outdoors', 'Games', 'Spiritual and Mental']
['Sports and Outdoors', 'Games', 'Spiritual and Mental', 'Performing Arts']
['Sports and Outdoors', 'Games', 'Spiritual and Mental']
['Sports and Outdoors', 'Games', 'Spiritual and Mental']
